# Project 4 Machine Learning Fairness Algorithms Evaluation

Group 6: 
* Huang, Xilin (xh2508@columbia.edu)
* Nguyen, Kieu-Giang (kn2521@columbia.edu) 
* Spade, Gabriel (gms2221@columbia.edu) 
* Wang, Yayuan (yw3548@columbia.edu)
* Xu, Jiapeng (jx2427@columbia.edu)


## 1. Introduction

​​In this project, we are comparing two methodologies introduced by [A2-Maximizing accuracy under fairness constraints (C-SVM and C-LR)](https://arxiv.org/abs/1507.05259) and [A7-Information Theoretic Measures for Fairness-aware Feature selection (FFS)](https://arxiv.org/abs/2106.00772) to obtain a better understanding of the trade-off between accuracy and fairness. The criterion we use to quantify fairness is calibration, meaning that the prediction accuracy of the protected group ought to be equal to the accuracy of the unprotected group. 

We examine all the algorithms on [COMPAS Dataset](https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis) to evaluate their performance. COMPAS dataset contains criminal history and demographic information. In this case, the race of each individual is our protected/sensitive attribute (African-American=0, Caucasian = 1). The binary target label (y) we are interested in is whether the individual was arrested for a crime within 2 years of release. 


In summary, according to the *A7-Information Theoretic Measures for Fairness-aware Feature Selection*, we select and encode our variables as follows: 

- Binary class label (y): <code>two_year_recid</code>, indicating whether the individual was arrested for a crime within 2 years of release
- Binary sensitive attribute: <code>race</code>, African-American=0, Caucasian = 1
- Other features: 
    *  <code>sex</code>: female = 0, male = 1
    *  <code>age</code>: less than 25 = 0, age 25~45 = 1, age older than 45 = 2
    *  <code>c_charge_degree</code>: misdemeanor = 0, felony = 1
    *  <code>priors_count</code>: none = 0, 1~3 times = 1, more than 3 = 2
    *  <code>length_of_stay</code>: normalized time elapsed from in jail until out of jail


## 2. Preparation
### 2.1 Data Cleansing and Processing

In [1]:
import numpy as np
import pandas as pd
import itertools
import copy
import math
from datetime import datetime, timedelta

In [2]:
df_raw = pd.read_csv('../data/compas-scores-two-years.csv')    # load the data

In [3]:
length_of_stay = pd.to_datetime(df_raw["c_jail_out"]) - pd.to_datetime(df_raw["c_jail_in"])    # calculate length of stay (days)
df = df_raw
df["length_of_stay"] = length_of_stay.astype('timedelta64[h]')/24
df = df[["two_year_recid","race","sex","age","c_charge_degree","priors_count","length_of_stay"]]   # select features we are interested in
df = df.dropna()         # drop NA
df = df.loc[(df["length_of_stay"] > 0) &                                         # length of stay should be positive
       ((df["race"]== "African-American") | (df["race"]== "Caucasian"))]         # individuals who are African american or Caucasian
df1 = df
df.head()

,two_year_recid,race,sex,age,c_charge_degree,priors_count,length_of_stay
1,1,African-American,Male,34,F,0,10.041667
2,1,African-American,Male,24,F,4,1.083333
6,1,Caucasian,Male,41,F,14,6.291667
8,0,Caucasian,Female,39,M,0,2.916667
9,1,Caucasian,Male,21,F,1,0.958333


In [4]:
# encode features
df['race'] = df['race'].apply(lambda race: 0 if race == 'African-American' else 1)
df['sex'] = df['sex'].apply(lambda sex: 0 if sex == 'Female' else 1)
df['age'] = df['age'].apply(lambda age: 0 if age < 25 else (2 if age > 45 else 1))
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda c_charge_degree: 0 if c_charge_degree == 'M' else 1)
df['priors_count'] = df['priors_count'].apply(lambda priors_count: 0 if priors_count == 0 else (2 if priors_count > 3 else 1))
#df['length_of_stay'] = pd.cut(df['length_of_stay'], bins = [0, 7, 90, np.inf], labels = [0, 1, 2])
df['length_of_stay'] = (df['length_of_stay'] - df['length_of_stay'].mean())/df['length_of_stay'].std()
df.head()

,two_year_recid,race,sex,age,c_charge_degree,priors_count,length_of_stay
1,1,0,1,1,1,0,-0.187151
2,1,0,1,0,1,2,-0.356541
6,1,1,1,1,1,2,-0.258059
8,0,1,0,1,0,0,-0.321875
9,1,1,1,0,1,1,-0.358905


### 2.1 Training and Test Sets

In [5]:
df_shuffled = df.sample(frac=1, random_state=1)      # shuffled the dataset
i = int(len(df_shuffled) * 0.7)         # use 70% as training set
train = df_shuffled[:i]                 # training set
test = df_shuffled[i:]                  # test set

label = "two_year_recid"
sensitive = "race"
features = ["sex","age","c_charge_degree","priors_count","length_of_stay"]

In [6]:
x_train, y_train, race_train = train[features], train[label].to_numpy(), train[sensitive]
x_test, y_test, race_test = test[features], test[label].to_numpy(), test[sensitive]

## 3. Baseline (Without Constraints)
### 3.1 Logistic Regression

Before fitting the logistic regression, let’s first define the function to calculate calibration(%) which could be understood as accuracy difference between two race groups.

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
# define the function for calculating calibration
def MyCalibration(sensitive_attr, y_pred, y_true):
    cau_index = np.where(sensitive_attr == 1)[0]           # index for Caucasians
    african_index = np.where(sensitive_attr == 0)[0]       # index for African-Americans
    
    y_pred_cau = y_pred[cau_index]           # Caucasians
    y_true_cau = y_true[cau_index] 
    Acc_cau = sum(y_pred_cau == y_true_cau)/len(y_pred_cau)

    y_pred_african = y_pred[african_index]   # African-Americans
    y_true_african = y_true[african_index]
    Acc_african = sum(y_pred_african == y_true_african)/len(y_pred_african)

    calibration = (Acc_cau - Acc_african)*100
    return(calibration)

In [9]:
logReg1 = LogisticRegression(random_state = 0).fit(x_train, y_train)   # logistic regression without constraints

In [10]:
summary_logReg = {"Methods": ["LR", "LR"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [logReg1.score(x_train, y_train)*100, logReg1.score(x_test, y_test)*100],
              "Calibration(%)": [MyCalibration(race_train, logReg1.predict(x_train), y_train),
                                 MyCalibration(race_test, logReg1.predict(x_test), y_test)]}

pd.DataFrame(summary_logReg)

,Methods,Set,Accuracy (%),Calibration(%)
0,LR,Train,66.307345,0.016413
1,LR,Test,66.257310,0.502892


### 3.2 Support Vector Machine (SVM) 

In [11]:
from sklearn.svm import SVC

In [12]:
svm_model1 = SVC(kernel = 'linear', probability = True, random_state = 0)
svm_model1 = svm_model1.fit(x_train, y_train)

In [13]:
summary_svm = {"Methods": ["SVM", "SVM"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [svm_model1.score(x_train, y_train)*100, svm_model1.score(x_test, y_test)*100],
              "Calibration(%)": [MyCalibration(race_train, svm_model1.predict(x_train), y_train),
                                 MyCalibration(race_test, svm_model1.predict(x_test), y_test)]}

pd.DataFrame(summary_svm)

,Methods,Set,Accuracy (%),Calibration(%)
0,SVM,Train,66.006518,0.731009
1,SVM,Test,64.561404,-0.431792


## 4. Implementation of Method 1 (A2)
### 4.1 Logistic Regression With Fairness Constraints


In [14]:
import sys
sys.path.insert(0, '../lib/')
import loss_function as lf
import clr as clr

In [15]:
# Use logistic loss function to compute the loss
loss_function = lf.logistic_loss

# Set max_iter = -1 to take as many iterations as we want
max_iter = -1

# Set apply_fairness_constraints to be true
apply_fairness_constraints = 1 

# Sensitive attributes
sensitive_attrs = ['race']

# Covariance threshold
sensitive_attrs_to_cov_thresh = {'race': 0}

x_control_train = {'race': race_train}
x_control_test = {'race': race_test}

In [16]:
CLR = clr.LR()
w = CLR.train_model(x_train, y_train, x_control_train, loss_function, max_iter, apply_fairness_constraints, sensitive_attrs, sensitive_attrs_to_cov_thresh)
clr_predict_y_train = np.sign(np.dot(x_train,w))
clr_predict_y_test = np.sign(np.dot(x_test,w))
clr_train_accuracy = sum(clr_predict_y_train == y_train)/len(y_train)
clr_test_accuracy = sum(clr_predict_y_test == y_test)/len(y_test)

In [17]:
summary_clr = {"Classifier": ["C-LR", "C-LR"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [clr_train_accuracy*100, clr_test_accuracy*100],
               "Calibration (%)": [MyCalibration(race_train, clr_predict_y_train, y_train), MyCalibration(race_test, clr_predict_y_test, y_test)]}
pd.DataFrame(summary_clr)

,Classifier,Set,Accuracy (%),Calibration (%)
0,C-LR,Train,47.630985,-13.727658
1,C-LR,Test,47.076023,-10.153752


### 4.2 Support Vector Machine (SVM) With Fairness Constraints

In [18]:
import csvm as csvm


In [19]:
# Use hinge loss function to compute the loss
loss_function = lf.hinge_loss

# Penalty
C = 1

# Lambda
lamb = 1
# Number of epochs
epochs = 1000
# Learning rate 
lr = 0.1  

# Gamma value strongly affect the accuracy, higher gamma will cause a huge decrease in accuracy
# To ensure our accuracy, we need to set it at a low value. In this case, we set gamma = 0
gamma = 0

In [20]:
CSVM = csvm.SVM()
w = CSVM.train_model(x_train, y_train, x_control_train, loss_function, C, max_iter, lamb, epochs, lr, apply_fairness_constraints, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma=0)
csvm_predict_y_train = np.sign(np.dot(x_train, w))
csvm_predict_y_test = np.sign(np.dot(x_test, w))
csvm_train_accuracy = sum(csvm_predict_y_train == y_train)/len(y_train)
csvm_test_accuracy = sum(csvm_predict_y_test == y_test)/len(y_test)

Running custom model


In [21]:
summary_c_svm = {"Classifier": ["C-SVM", "C-SVM"],
                "Set": ["Train", "Test"],
                "Accuracy (%)": [csvm_train_accuracy*100, csvm_test_accuracy*100],
                "Calibration (%)": [MyCalibration(race_train, csvm_predict_y_train, y_train), MyCalibration(race_test, csvm_predict_y_test, y_test)]}
pd.DataFrame(summary_c_svm)

,Classifier,Set,Accuracy (%),Calibration (%)
0,C-SVM,Train,48.032088,-13.779814
1,C-SVM,Test,47.777778,-9.571395


## 5. Implementation of Method 2 (A7)

In [22]:
# encode features as proposed by A7
df1 = df_raw
df1["length_of_stay"] = length_of_stay.astype('timedelta64[h]')/24
df1 = df1[["two_year_recid","race","sex","age","c_charge_degree","priors_count","length_of_stay"]]   # select features we are interested in
df1 = df1.dropna()         # drop NA
df1 = df1.loc[(df1["length_of_stay"] > 0) &                                         # length of stay should be positive
       ((df1["race"]== "African-American") | (df1["race"]== "Caucasian"))]         # individuals who are African american or Caucasian


df1['race'] = df1['race'].apply(lambda race: 0 if race == 'African-American' else 1)
df1['sex'] = df1['sex'].apply(lambda sex: 0 if sex == 'Female' else 1)
df1['age'] = df1['age'].apply(lambda age: 0 if age < 25 else (2 if age > 45 else 1))
df1['c_charge_degree'] = df1['c_charge_degree'].apply(lambda c_charge_degree: 0 if c_charge_degree == 'M' else 1)
df1['priors_count'] = df1['priors_count'].apply(lambda priors_count: 0 if priors_count == 0 else (2 if priors_count > 3 else 1))
df1['length_of_stay'] = pd.cut(df1['length_of_stay'], bins = [0, 7, 90, np.inf], labels = [0, 1, 2])

In [23]:
df_shuffled1 = df1.sample(frac=1, random_state=1)      # shuffled the dataset
i = int(len(df_shuffled1) * 0.7)         # use 70% as training set
train1 = df_shuffled1[:i]                 # training set
test1 = df_shuffled1[i:]                  # test set

label = "two_year_recid"
sensitive = "race"
features = ["sex","age","c_charge_degree","priors_count","length_of_stay"]

In [24]:
x_train1, y_train1, race_train1 = train1[features], train1[label].to_numpy(), train1[sensitive]
x_test1, y_test1, race_test1 = test1[features], test1[label].to_numpy(), test1[sensitive]

In [25]:
def unique_values_in_arrays(arr):
    
    # arr: n * m matrix
    # each elements in unique_vals contains unique values of ith column in arr

    unique_vals = []
    for i in range(arr.shape[1]):
        unique_vals.append(np.unique(arr[:, i]).tolist())
    return unique_vals

def power_set(seq):
    
    #This function create a generator that contains all subsets of seq
    
    if len(seq) <= 1:
        yield seq
        yield []
    else: 
        for i in power_set(seq[1:]):
            yield [seq[0]] + i
            yield i
            
def unique_info(array_1, array_2):
    assert array_1.shape[0] == array_2.shape[0]
    
    n_rows = array_1.shape[0]
    n_col_array_1 = array_1.shape[1]
    
    concated_array = np.concatenate((array_1, array_2), axis=1)
    unique_array = unique_values_in_arrays(concated_array)
    cartesian_product = list(itertools.product(*unique_array))
    
    # IQ(T; R1|R2) = ∑t,r1,r2 QT ,R1,R2 (t, r1, r2) log((QT |R1,R2 (t|r1,r2))/ (QT |R2 (t|r2))) 
    
    IQ = 0
    for i in cartesian_product:
        r1_r2 = len(np.where((concated_array == i).all(axis=1))[0]) / n_rows
        r1 = len(np.where((array_1 == i[:n_col_array_1]).all(axis=1))[0]) / n_rows
        r2 = len(np.where((array_2 == i[n_col_array_1:]).all(axis=1))[0]) / n_rows
        
        if r1_r2 == 0 or r1 == 0 or r2 == 0:
            IQ_iter = 0
        else:
            IQ_iter = r1_r2 * np.log(r1_r2 / r1) / r1
        IQ += np.abs(IQ_iter)
    return IQ

def unique_info_conditional(array_1, array_2, conditional):
    assert (array_1.shape[0] == array_2.shape[0]) and (array_1.shape[0] == conditional.shape[0])
    
    n_rows = array_1.shape[0]
    n_col_array_1 = array_1.shape[1]
    n_col_array_2 = array_2.shape[1]
    
    concated_array_2_conditional = np.concatenate((array_2, conditional), axis=1)
    concated_array_all           = np.concatenate((array_1, concated_array_2_conditional), axis=1)
    unique_array                 = unique_values_in_arrays(concated_array_all)
    cartesian_product = list(itertools.product(*unique_array))
    
    IQ = 0
    for i in cartesian_product:
        r1_r2 = len(np.where((concated_array_all == i).all(axis=1))[0]) / n_rows
        r1    = len(np.where((array_1 == i[:n_col_array_1]).all(axis=1))[0]) / n_rows
        r2    = len(np.where((concated_array_all[:, n_col_array_1: -n_col_array_2] == i[n_col_array_1: -n_col_array_2]).all(axis=1))[0]) / n_rows
        
        try:
            r1_given_r2 = len(np.where((concated_array_all[:, :n_col_array_1] == i[ :n_col_array_1]).all(axis=1) & (concated_array_all[:, -n_col_array_2:] == i[-n_col_array_2:]).all(axis=1))[0]) / len(np.where((concated_array_all[:, -n_col_array_2:] == i[-n_col_array_2:]).all(axis=1))[0])
        except ZeroDivisionError:
            r1_given_r2 = 0
        
        if r1_r2 == 0 or r1 == 0 or r2 == 0 or r1_given_r2 == 0:
            IQ_iter = 0
        else:
            IQ_iter = r1_r2 * np.log(r1_r2 / r2) / r1_given_r2
        IQ += np.abs(IQ_iter)
    return IQ

def accuracy_coef(y, x_s, x_s_c, A):
    # Calculate accuracy coefficient
    conditional = np.concatenate((x_s_c, A), axis=1)
    return unique_info_conditional(y, x_s, conditional)

def discrimination_coef(y, x_s, A):
    # Calculate discrimination coefficient
    x_s_a = np.concatenate((x_s, A), axis=1)
    return unique_info(y, x_s_a) * unique_info(x_s, A) * unique_info_conditional(x_s, A, y)

def marginal_accuracy_coef(y_train, x_train, A, set_tracker):
    
    #compute  marginal accuracy coefficient
    
    n_features = x_train.shape[1]
    feature_idx = list(range(n_features))
    feature_idx.pop(set_tracker)
    power_set_features = [x for x in power_set(feature_idx) if len(x) > 0]
    
    shapley_value =0
    for sc_idx in power_set_features:
            coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)

            # Compute v(T ∪ {i}) 
            idx_xs_ui = copy.copy(sc_idx) # create copy of subset list
            idx_xs_ui.append(set_tracker) # append feature index
            idx_xsc_ui = list(set(list(range(n_features))).difference(set(idx_xs_ui))) # compliment of x_s
            vTU = accuracy_coef(y_train.reshape(-1, 1), x_train[:, idx_xs_ui], x_train[:, idx_xsc_ui], A.reshape(-1, 1))

             # Compute v(T)
            idx_xsc = list(range(n_features))
            idx_xsc.pop(set_tracker)
            idx_xsc = list(set(idx_xsc).difference(set(sc_idx)))
            vT = accuracy_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], x_train[:, idx_xsc], A.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal
    return shapley_value

def marginal_discrimination_coef(y_train, x_train, A, set_tracker):
    
    # compute marginal discrimination coefficient
    
    n_features = x_train.shape[1]
    feature_idx = list(range(n_features))
    feature_idx.pop(set_tracker)
    power_set_features = [x for x in power_set(feature_idx) if len(x) > 0]
    
    shapley_value =0
    for sc_idx in power_set_features:
            coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)

            # Compute v(T ∪ {i}) 
            idx_xs_ui = copy.copy(sc_idx) # create copy of subset list
            idx_xs_ui.append(set_tracker) # append feature index
            vTU = discrimination_coef(y_train.reshape(-1, 1), x_train[:, idx_xs_ui], A.reshape(-1, 1))

            # Compute v(T)
            vT = discrimination_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], A.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal
    return shapley_value

In [26]:
%%time
shapley_acc = []
shapley_disc = []
for i in range(5):
    acc_i = marginal_accuracy_coef(y_train1, x_train1.to_numpy(), race_train1.to_numpy(), i)
    disc_i = marginal_discrimination_coef(y_train1, x_train1.to_numpy(), race_train1.to_numpy(), i)
    
    
    shapley_acc.append(acc_i)
    shapley_disc.append(disc_i)

# DataFrame to compare shapely values
feature_names = ["Sex","Age","Charge Degree","Priors Count","Length of Stay"]
shapley_df = pd.DataFrame(list(zip(feature_names, shapley_acc, shapley_disc)),
                          columns=["Feature", "Accuracy",'Discrimination'])
shapley_df

CPU times: user 15.9 s, sys: 176 ms, total: 16 s
Wall time: 15.9 s


,Feature,Accuracy,Discrimination
0,Sex,0.999906,37894.189102
1,Age,1.202882,47850.121441
2,Charge Degree,1.069162,38978.545863
3,Priors Count,1.231735,48379.871765
4,Length of Stay,1.139896,46926.923113


**Conclusion:** \
According to the algorithm in *'Information Theoretic Measures for Fairness-aware Feature selection (FFS)'*, we calculated the marginal accuracy coefficient and the marginal discrimination coefficient. The result above shows that **Age** and **Priors Counts** have the most significant impact on accuracy and strongest proxies for discrimination simultaneously, which is aligned with the conclusion in the paper A7. Therefore, simply dropping **Age** or **Priors Counts** may highly influence the model accuracy and calibration at the same time.

To make a wiser decision for feature selection, we calculated two fairness utility score introduced by the paper A7 under two different hyperparameter $\alpha$ which trade-off between accuracy and discrimination.

In [27]:
def fairness_utility_score(Accuracy, Discrimination, alpha):
    scores = []
    for i in range(5):
        score = Accuracy[i] - alpha * Discrimination[i]
        scores.append(score)
    return scores

**(1) $\alpha$ = 0.000001**

In [28]:
alpha_1 = pd.DataFrame(list(zip(feature_names, shapley_acc, shapley_disc, fairness_utility_score(shapley_df['Accuracy'], shapley_df['Discrimination'], 0.000001))),
                       columns=["Feature", "Accuracy",'Discrimination', 'F_score'])
alpha_1

,Feature,Accuracy,Discrimination,F_score
0,Sex,0.999906,37894.189102,0.962012
1,Age,1.202882,47850.121441,1.155032
2,Charge Degree,1.069162,38978.545863,1.030184
3,Priors Count,1.231735,48379.871765,1.183355
4,Length of Stay,1.139896,46926.923113,1.092969


**(1) $\alpha$ = 0.0001**

In [29]:
alpha_2 = pd.DataFrame(list(zip(feature_names, shapley_acc, shapley_disc, fairness_utility_score(shapley_df['Accuracy'], shapley_df['Discrimination'], 0.0001))),
                       columns=["Feature", "Accuracy",'Discrimination', 'F_score'])
alpha_2

,Feature,Accuracy,Discrimination,F_score
0,Sex,0.999906,37894.189102,-2.789513
1,Age,1.202882,47850.121441,-3.582130
2,Charge Degree,1.069162,38978.545863,-2.828692
3,Priors Count,1.231735,48379.871765,-3.606252
4,Length of Stay,1.139896,46926.923113,-3.552796


**Conclusion:**\
Normally, we would like to remove the feature with the lowest fairness-utility score. Under the circumstances with different $\alpha$, the featues with the lowest fairness-utility score are:

1) When  𝛼=0.000001, **Sex**;\
2) When  𝛼=0.0001, **Priors Count**;

A smaller $\alpha$ means we care more about the accuracy, and A bigger $\alpha$ means we care more about the discrimination effect. Therefore,  if we care about maintaining very high accuracy, we can choose to remove **Sex**; If we want to minimize the discrimination effect, we can choose to remove **Prior Counts**.

In the next step, we will first fit the dataset with the whole five features on our baseline models, and then remove one feature from **Gender**, **Length of Stay**, **Age**, and **Prior Counts** respectively and compare their accuracy and calibration.

### 5.1 Logistic Regression After Features Selection

In [30]:
Accuracy_lr = []
Calibration_lr = []

for i in ['None'] + features:
    if i == 'None':
        logReg = LogisticRegression(random_state = 0).fit(x_train1, y_train1)
        Accuracy_lr.append(logReg.score(x_test1, y_test1)*100)
        Calibration_lr.append(MyCalibration(race_test1, logReg.predict(x_test1), y_test1))
    else:
        x_train_subset = x_train1.drop([i],axis = 1)
        x_test_subset = x_test1.drop([i],axis = 1)
        logReg = LogisticRegression(random_state = 0).fit(x_train_subset, y_train1)
        Accuracy_lr.append(logReg.score(x_test_subset, y_test1)*100)
        Calibration_lr.append(MyCalibration(race_test1, logReg.predict(x_test_subset), y_test1))
        
col_names = ['None'] + feature_names
Conclusion_lr = pd.DataFrame(list(zip(col_names, Accuracy_lr, Calibration_lr)),
                          columns=["Eliminating Feature", "Accuracy (%)", "Calibration (%)"])
Conclusion_lr

,Eliminating Feature,Accuracy (%),Calibration (%)
0,None,65.672515,0.716337
1,Sex,65.906433,1.074866
2,Age,62.280702,4.025879
3,Charge Degree,65.614035,0.565051
4,Priors Count,59.122807,4.241342
5,Length of Stay,66.666667,0.575435


### 5.2 Support Vector Machine (SVM) After Features Selection

In [31]:
Accuracy_svm = []
Calibration_svm = []

for i in ['None'] + features:
    if i == 'None':
        svm_model = SVC(kernel = 'linear', probability = True, random_state = 0)
        svm_model = svm_model.fit(x_train1, y_train1)
        Accuracy_svm.append(svm_model.score(x_test1, y_test1)*100)
        Calibration_svm.append(MyCalibration(race_test1, svm_model.predict(x_test1), y_test1))
    else:
        x_train_subset = x_train1.drop([i],axis = 1)
        x_test_subset = x_test1.drop([i],axis = 1)
        svm_model = SVC(kernel = 'linear', probability = True, random_state = 0)
        svm_model = svm_model.fit(x_train_subset, y_train1)
        Accuracy_svm.append(svm_model.score(x_test_subset, y_test1)*100)
        Calibration_svm.append(MyCalibration(race_test1, svm_model.predict(x_test_subset), y_test1))
        
Conclusion_svm = pd.DataFrame(list(zip(col_names, Accuracy_svm, Calibration_svm)),
                              columns=["Eliminating Feature", "Accuracy (%)", "Calibration (%)"])
Conclusion_svm

,Eliminating Feature,Accuracy (%),Calibration (%)
0,None,65.847953,0.923580
1,Sex,66.491228,1.108036
2,Age,59.532164,4.313884
3,Charge Degree,66.140351,-0.046294
4,Priors Count,59.122807,4.241342
5,Length of Stay,63.859649,-0.274305


**Conclusion:**\
According to the marginal accuracy coefficient and the marginal discrimination coefficient, we expect the model accuracy and calibration will decrease simultaneously after dropping **Age** and **Priors Count**. However, the fact is the model accuracy did decrease in both models but the calibration increased in contrast, which is not aligned with the theory. From the table above, the best model of linear regression is the model without the feature '**Length of Stay**' and the best model of support vector machine is the model without the feature '**Charge Degree**' when we take accuracy and discrimination effects into account at the same time. It shows a pretty high accuracy with the lowest calibration.

There are several possible explanations for the fluctuation of the calibration:

1) Fairness-utility scores are pretty close to each other under our several choices of hyperparameters, so that we cannot guarantee removing the feature with the lowest score is always better than removing another feature also with a very low score.

2) The marginal accuracy coefficient and marginal discrimination coefficient of the five features are on the same scale. There are no features with outlying discrimination scores or accuracy scores. Therefore we didn't observe remarkable differences under our evaluation metrics.

3) Number of test data, and the proportion of train/test/validation may matter.

In [32]:
# Logistic regression without "Length of Stay"

x_train_subset2 = x_train1.drop(["length_of_stay"],axis = 1)
x_test_subset2 = x_test1.drop(["length_of_stay"],axis = 1)
logReg_selection = LogisticRegression(random_state = 0).fit(x_train_subset2, y_train1)

In [33]:
# Logistic regression without "Length of Stay"

x_train_subset3 = x_train1.drop(["c_charge_degree"],axis = 1)
x_test_subset3 = x_test1.drop(["c_charge_degree"],axis = 1)
svm_model_selection = SVC(kernel = 'linear', probability = True, random_state = 0)
svm_model_selection = svm_model.fit(x_train_subset3, y_train1)

## 6. Evaluation

In [34]:
# summary table for test results

summary_total = {"Methods": ["LR", "SVM", "C-LR", "C-SVM", "S-LR", "S-SVM"], 
              "Accuracy (%)": [logReg1.score(x_test, y_test)*100, svm_model1.score(x_test, y_test)*100,
                               clr_test_accuracy*100, csvm_test_accuracy*100,
                               logReg_selection.score(x_test_subset2, y_test1)*100,
                               svm_model_selection.score(x_test_subset3, y_test1)*100],
              "Calibration(%)": [MyCalibration(race_test, logReg1.predict(x_test), y_test),
                                 MyCalibration(race_test, svm_model1.predict(x_test), y_test),
                                 MyCalibration(race_test, clr_predict_y_test, y_test), 
                                 MyCalibration(race_test, csvm_predict_y_test, y_test),
                                 MyCalibration(race_test, logReg_selection.predict(x_test_subset2), y_test1),
                                 MyCalibration(race_test, svm_model_selection.predict(x_test_subset3), y_test1)]}
pd.DataFrame(summary_total)    


,Methods,Accuracy (%),Calibration(%)
0,LR,66.257310,0.502892
1,SVM,64.561404,-0.431792
2,C-LR,47.076023,-10.153752
3,C-SVM,47.777778,-9.571395
4,S-LR,66.666667,0.575435
5,S-SVM,66.140351,-0.046294


Between methods proposed by A2 and A7, the feature selection approach does a better job of increasing accuracy and lowering the calibration. In addition, With similar accuracy scores, S-SVM is more promising regarding trade-off accuracy and fairness considerations. 